In [1]:
import warnings
warnings.filterwarnings('ignore')

In [93]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [9]:
def read_in_the_file(file_name):
    #read in subject file
    with open(file_name) as f:
         file_ = [line.strip() for line in f]
read_in_the_file('target.txt')

['have a great day?',
 'is there anything else i can help you with today?',
 'you are welcome.',
 'great.',
 'thank you.',
 'is there anything else i can assist you with?',
 'okay.',
 'is there anything else i can help you with?',
 'what is your account number?',
 'what is your order number?',
 'you are very welcome.',
 'ok great.',
 'is there anything else i can assist you with today?',
 'i am sorry to hear that.',
 'what is your new address?',
 'thanks.',
 'i am happy to look into this for you.',
 'have a safe flight?',
 'thank you for that information.',
 'does that work for you.',
 'does that work.',
 'what is the order number?',
 'his.',
 'ok awesome.',
 'i am happy to assist you.',
 'enjoy the rest of your day.',
 'have a great day further?',
 'no problems.',
 'have you tried reporting the system?',
 'what is your cell number?',
 'i can assist you.',
 'have a wonderful day?',
 'sure.',
 'i can help you.',
 'great news.',
 'have a good day?',
 'i can help you with this.',
 'which 

In [ ]:
'''
preprocessing stage
-------------------
    - spell check
    - lower casing
    - punctuation removal
    - number removal?
    - html removal
    - snowball, port stemming 
'''

def remove_punc(t_string):
    check = lambda char: ' ' if char in set(punctuation) else char
    return ''.join(map(check, list(t_string)))

def clean(name):
    #remove spaces
    name = name.strip()
    
    #strip out
    name = name.replace('\"', '')    
    name = encode(name)    
    
    return name

In [38]:
def build_vectorizer(corpus, model_type='bag of words', ngrams=1, tokenizer='char'):
    '''
    *add word2vec
    '''
    #think of params
    params = {
        'analyzer': tokenizer,
        'ngram_range' : (1, ngrams)
    }

    if model_type == 'bag of words':
        vectorizer = CountVectorizer(**params)
    elif model_type ==  'tfidf':
        vectorizer = TfidfVectorizer(**params)
    
    self.vector_space = vectorizer.fit_transform(corpus) 
    self.vectorizer = vectorizer 

In [132]:
from sklearn.neighbors import NearestNeighbors

vector_space, vectorizer = build_vectorizer(file_, model_type='tfidf', ngrams=1)
nn = NearestNeighbors(n_jobs=-1, algorithm='brute')
nn.fit(vector_space)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=-1, n_neighbors=5, p=2, radius=1.0)

In [150]:
def predict(new_data_pt, radius_threshold=.25):
    '''
    not sure how to find the optimal threshold here
    '''
    
    pt = vectorizer.transform([new_data_pt])
    
    #how to find optimal radius?
    distance_from_origin, indices = nn.radius_neighbors(pt, radius=radius_threshold)

    #unpacking
    distance_from_origin = distance_from_origin[0]
    indices = indices[0]
    
    grabbing_the_lines_from_file = [file_[index] for index in indices]
    
    return grabbing_the_lines_from_file

predict(file_[1])

['is there anything else i can help you with today?',
 'is there anything else i can help you with?',
 'is there anything else i can help you with a?',
 'can i help you with anything else?',
 'is there anything else can help you with today?',
 'can i help you with anything else today?',
 'is there anything i can help you with today?',
 'is there anything else i can help youth?',
 'is there anything else i can do to help you today?',
 'is there any thing else i can help you with today?',
 'is there anything i can help you with?',
 'is there anything else i can help you?',
 'is there anything else we can help you with today?',
 'is there anything else i cane help you with?',
 'is there anything use i can help you with today?',
 'is there anything else you can help me with?',
 'is there anything else i can help you it?',
 'i there anything else i can help you with today.',
 'is there anything else i can help you with patricia?',
 'is there anything else i can help with today?',
 'is there

In [149]:
def find_all_duplicates():
    #find all duplicates
    all_dups_dict = {idx : predict(line) for idx, line in enumerate(file_)}
    return all_dups_dict

find_all_duplicates()

{0: ['have a great day?',
  'have a a great day?',
  'have great day?',
  'have a great day great?'],
 1: ['is there anything else i can help you with today?',
  'is there anything else i can help you with?',
  'is there anything else i can help you with a?',
  'can i help you with anything else?',
  'is there anything else can help you with today?',
  'can i help you with anything else today?',
  'is there anything i can help you with today?',
  'is there anything else i can help youth?',
  'is there anything else i can do to help you today?',
  'is there any thing else i can help you with today?',
  'is there anything i can help you with?',
  'is there anything else i can help you?',
  'is there anything else we can help you with today?',
  'is there anything else i cane help you with?',
  'is there anything use i can help you with today?',
  'is there anything else you can help me with?',
  'is there anything else i can help you it?',
  'i there anything else i can help you with tod

In [159]:
from sklearn.neighbors import LSHForest

X_test = vectorizer.transform([file_[368]])

lshf = LSHForest(random_state=42)
lshf.fit(vector_space)  
LSHForest(min_hash_match=4, n_candidates=50, n_estimators=10,
          n_neighbors=5, radius=1.0, radius_cutoff_ratio=0.9,
          random_state=42)

distances, indices = lshf.radius_neighbors(X_test, radius=.05)

print distances                                        
print indices


[ array([ -2.22044605e-16,   1.19601122e-02,   1.20245760e-02,
         2.57881884e-02,   2.68411418e-02,   3.42908780e-02,
         4.24374459e-02])]
[array([ 368, 8445, 7961, 2494, 1109, 6914, 6057])]
